In [64]:
import json
import geojson
import math
import heapq


# Data processing parse data from airports.json data
def parse_airport_data(filename):
    with open(filename, 'r') as file:
        data = json.load(file)

    airport = {}
    for feature in data['features']:
        property = feature['properties']
        coordinate = feature['geometry']['coordinates']
        iata = property.get('iata_code', None)

        if iata:
            airport[iata]={
                'name': property.get('name', None),
                "longitude": coordinate[0],
                "latitude": coordinate[1]
            }
    return airport


# calculate the haversine distance
def haversine_distance(lon1, lat1, lon2, lat2):
    R = 6371  # Earth's radius in kilometers

    # Convert to radians
    # Extract first element if they are lists
    lon1 = lon1[0] if isinstance(lon1, list) else lon1
    lat1 = lat1[0] if isinstance(lat1, list) else lat1
    lon2 = lon2[0] if isinstance(lon2, list) else lon2
    lat2 = lat2[0] if isinstance(lat2, list) else lat2

    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])


    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.asin(math.sqrt(a))

    return R * c

# Define the graph to create and adjacent list with airports connected by the routes shorter
# than the max_range
def construct_graph(airport, max_range):
    graph = {iata: [] for iata in airport}

    for iata1, airport1 in airport.items():
        for iata2, airport2 in airport.items():
            if iata1!=iata2:
                distance = haversine_distance(airport1['longitude'], airport1['latitude'],
                              airport2['longitude'], airport2['latitude'])

                if distance < max_range:
                    graph[iata1].append((iata2, distance))

    return graph


# Define the dijstra algorithm
def dijkstra(graph, start, end):
    # cost, airport, path
    pq = [(0, start, [])]
    visited = set()

    while pq:
        cost, airport, path = heapq.heappop(pq)
        if airport in visited:
            continue
        visited.add(airport)
        path = path + [airport]

        if airport == end:
            return path, cost
        
        for neighbor, weight in graph.get(airport, []):
            if neighbor not in visited:
                heapq.heappush(pq, (cost + weight, neighbor, path))

    return None, float('inf')


# Define A* ALGORITHM
def a_aligo(graph, airport, start, end):
    def heuristic(a, b):
        return haversine_distance(airport[a]['longitude'], airport[a]['latitude'],
                                     airport[b]['longitude'], airport[b]['latitude'])
    
    # Getting cost + heuristic, airport, path
    pq = [(0, start, [])]
    visited = set()

    while pq:
        cost, current_airport, path = heapq.heappop(pq)
        if current_airport in visited:
            continue
        visited.add(current_airport)
        path = path + [current_airport]

        if current_airport == end:
            return path, cost
        
        for neighbor, weight in graph.get(current_airport, []):
            if neighbor not in visited:
                total_cost = cost + weight + heuristic(neighbor, end)
                heapq.heappush(pq, (total_cost, neighbor, path))

    return None, float('inf')


# Find the best route
def best_route(filename, start, end, max_range, aligo="dijkstra"):
    airports = parse_airport_data(filename)

    if start not in airports or end not in airports:
        return f"Invalid IATA codes"
    
    graph = construct_graph(airports, max_range)

    if aligo == "dijkstra":
        path, distance = dijkstra(graph, start, end)
    else:
        path, distance = a_aligo(graph, airports, start, end)

    if path:
        return f"The route for {aligo} is {'--->'.join(path)}, Total Distance: {distance:.2f} Km"
    else:
        return "No valide route Found"
    

# # Examples:
# if __name__ == "__main__":
    

# Example Usage
if __name__ == "__main__":
    filename = "airports.geojson"
    start = "CDG"  # Example: Charles de Gaulle
    end = "NRT"  # Example: Narita Int'l
    max_range = 5000  # Example aircraft range
    
    print(best_route(filename, start, end, max_range, "dijkstra"))
    print(best_route(filename, start, end, max_range, "astar"))






# airport = parse_airport_data('airports.geojson')
# airport[a]['longitude']

# graph = {iata: [] for iata in airport}

# for iata1, airport1 in airport.items():
#     for iata2, airport2 in airport.items():
#         if iata1!=iata2:
#             print(f"Checking values: lon1={airport1['longitude']}, lat1={airport1['latitude']}, lon2={airport2['longitude']}, lat2={airport2['latitude']}")
            # distance = haversine_distance(airport1['longitude'], airport1['latitude'],
            #                 airport2['longitude'], airport2['latitude'])
# airport['longitude']
# {iata: [] for iata in airport}
# for iata1, aiport1 in airport.items():
#     for iata2, airport2 in airport.items():
#         print(iata1, aiport1)
#         print("_______________________")
#         print(iata2, airport2)
#         print("**********************")




# # Load GeoJSON data
# with open('airports.geojson') as f:
#     data = geojson.load(f)

# # Accessing GeoJSON features
# for feature in data['features']:
#     print(feature['geometry']['coordinates'][0])
#     # print(feature['properties'].get('iata_code'))
#     # print(feature['properties'].get('iata_code', None))



The route for dijkstra is CDG--->SGC--->VVO--->NRT, Total Distance: 9825.70 Km
The route for astar is CDG--->SGC--->OKD--->NRT, Total Distance: 16252.98 Km


In [70]:
import math

def security_robot(current_floor, emergencies, current_location):
    """
    Simulates a reflex agent for a security robot.
    
    Parameters:
    - current_floor: The floor where the robot is currently located.
    - emergencies: A dictionary containing detected emergencies and their locations.
                  Format: {'intruder': floor, 'fire': floor, 'gas_leak': floor, 'flood': floor}
    - current_location: The robot's current location on the floor (in meters from the origin).

    Returns:
    - A string describing the action taken by the robot and the quickest route.
    """
    # Priority order of emergencies
    priority_order = ['intruder', 'fire', 'gas_leak', 'flood']

    # Determine the highest priority emergency
    highest_priority_emergency = None
    emergency_floor = None
    for emergency in priority_order:
        if emergency in emergencies:
            highest_priority_emergency = emergency
            emergency_floor = emergencies[emergency]
            break  # Stop at the highest priority emergency

    # If no emergency is detected, continue patrolling
    if highest_priority_emergency is None:
        return f"Continue patrolling the building on floor {current_floor}"

    # Define known locations of stairs and elevator
    stairs_location = (1000, 1000)  # Stairs are 1km North and 1km East
    elevator_location = (500, 100)  # Corrected elevator location

    # Calculate Euclidean distances to stairs and elevator
    distance_to_stairs = math.sqrt((current_location[0] - stairs_location[0]) ** 2 +
                                   (current_location[1] - stairs_location[1]) ** 2)
    distance_to_elevator = math.sqrt((current_location[0] - elevator_location[0]) ** 2 +
                                     (current_location[1] - elevator_location[1]) ** 2)

    # Time required to reach stairs or elevator
    robot_speed = 3  # m/s
    time_to_stairs = distance_to_stairs / robot_speed
    time_to_elevator = distance_to_elevator / robot_speed

    # Time taken per floor for stairs and elevator
    elevator_time_per_floor = 30  # seconds per floor
    stairs_time_per_floor = 60  # seconds per floor

    # Compute total time using each route
    total_stairs_time = time_to_stairs + abs(emergency_floor - current_floor) * stairs_time_per_floor
    total_elevator_time = time_to_elevator + abs(emergency_floor - current_floor) * elevator_time_per_floor

    # Choose the fastest route
    if total_stairs_time < total_elevator_time:
        route = "stairs"
    else:
        route = "elevator"

    # Define action based on emergency type
    action_map = {
        "intruder": f"Emit alarm and move to location of intruder on floor {emergency_floor} using {route}",
        "fire": f"Call fire department and evacuate building on floor {emergency_floor} using {route}",
        "gas_leak": f"Turn off gas supply and evacuate building on floor {emergency_floor} using {route}",
        "flood": f"Turn off electricity supply and evacuate building on floor {emergency_floor} using {route}"
    }

    return action_map[highest_priority_emergency]


# Example Usage
if __name__ == "__main__":
    # Example 1: Intruder detected on floor 5
    emergencies = {'intruder': 5}
    current_location = (20, 100)  # Robot is 20m East and 100m North of the origin
    current_floor = 0
    print(security_robot(current_floor, emergencies, current_location))

    # Example 2: Fire detected on floor 3
    emergencies = {'fire': 3}
    current_location = (500, 500)  # Robot is 500m East and 500m North of the origin
    current_floor = 1
    print(security_robot(current_floor, emergencies, current_location))

    # Example 3: No emergency detected
    emergencies = {}
    current_location = (0, 0)  # Robot is at the origin
    current_floor = 2
    print(security_robot(current_floor, emergencies, current_location))


Emit alarm and move to location of intruder on floor 5 using elevator
Call fire department and evacuate building on floor 3 using elevator
Continue patrolling the building on floor 2


In [69]:
priority = ['intruder', 'fire', 'gas', 'flood']
emergencies = {'intruder': 5, 'fire': 2, 'gas': 3}
current_emergency = None
emergency_floor = None
for emergency in priority:
    if emergency in emergencies:
        current_emergency = emergency
        emergency_floor = emergencies[emergency]
        print(emergency_floor)


5
2
3


In [75]:
import math

def security_robot(current_floor, emergencies, current_location):
    # Priority order of emergencies
    priority_order = ['intruder', 'fire', 'gas_leak', 'flood']

    # Find the highest priority emergency
    selected_emergency = None
    emergency_floor = None
    for emergency in priority_order:
        if emergency in emergencies:
            selected_emergency = emergency
            emergency_floor = emergencies[emergency]
            break  # Stop at the highest priority emergency

    # If no emergency is detected, continue patrolling
    if selected_emergency is None:
        return f"Continue patrolling the building on floor {current_floor}"

    # Define known locations of stairs and elevator
    #  Stairs are 1000m North and 1000m East
    stairs_location = (1000, 1000)  

    # Corrected elevator location to have both East, west movement and 900m South
    elevator_location = (1000, 100)  

    # Euclidean distances to stairs and elevator - this will facto in the sounth and west
    distance_to_stairs = math.sqrt((current_location[0] - stairs_location[0]) ** 2 +
                                   (current_location[1] - stairs_location[1]) ** 2)
    distance_to_elevator = math.sqrt((current_location[0] - elevator_location[0]) ** 2 +
                                     (current_location[1] - elevator_location[1]) ** 2)

    # Time to reach stairs or elevator
    robot_speed = 3  # m/s
    time_to_stairs = distance_to_stairs / robot_speed
    time_to_elevator = distance_to_elevator / robot_speed

    # Time taken per floor for stairs and elevator {Seconds}
    elevator_time_per_floor = 30   
    stairs_time_per_floor = 60  

    # Compute total time using each route
    total_stairs_time = time_to_stairs + abs(emergency_floor - current_floor) * stairs_time_per_floor
    total_elevator_time = time_to_elevator + abs(emergency_floor - current_floor) * elevator_time_per_floor

    # Findfastest route
    if total_stairs_time < total_elevator_time:
        route = "stairs"
    else:
        route = "elevator"

    # Take action emergency type
    action_map = {
        "intruder": f"Emit alarm and move to location of intruder on floor {emergency_floor} using {route}",
        "fire": f"Call fire department and evacuate building on floor {emergency_floor} using {route}",
        "gas_leak": f"Turn off gas supply and evacuate building on floor {emergency_floor} using {route}",
        "flood": f"Turn off electricity supply and evacuate building on floor {emergency_floor} using {route}"
    }

    return action_map[selected_emergency]


# Example
if __name__ == "__main__":
    # 1: Intruder detected on floor 5
    emergencies = {'intruder': 5}
    current_location = (20, 100)  # Robot is 20m East and 100m North of the origin
    current_floor = 0
    print(security_robot(current_floor, emergencies, current_location))

    # 2: Fire detected on floor 3
    emergencies = {'fire': 3}
    current_location = (500, 500)  # Robot is 500m East and 500m North of the origin
    current_floor = 1
    print(security_robot(current_floor, emergencies, current_location))

    # 3: multiple emergency
    emergencies = {"intruder": 4, 'gas': 2, 'flood': 1}
    current_location = (200, 100)  # Robot is 200m East and 100m North of the origin
    current_floor = 3
    print(security_robot(current_floor, emergencies, current_location))

    # 4: No emergency detected
    emergencies = {}
    current_location = (0, 0)  # Robot is at the origin
    current_floor = 2
    print(security_robot(current_floor, emergencies, current_location))


Emit alarm and move to location of intruder on floor 5 using elevator
Call fire department and evacuate building on floor 3 using elevator
Emit alarm and move to location of intruder on floor 4 using elevator
Continue patrolling the building on floor 2


In [74]:
import gymnasium as gym
import numpy as np

map = ["SFFFHFFF", "FHFFFHFF", "FFFHFHFH", "HFFFHFFF", "FFFFHHFF", "HFFFFFFF", "FFHHHFHF", "FFFHFFFF"]
env = gym.make('FrozenLake-v1', desc=map, map_name="8x8", is_slippery=False, render_mode="human")

# This function is a placeholder--you will have to replace it.
def best_next_action(observation):
    return 1

observation, info = env.reset()
for _ in range(1000):
    action = best_next_action(observation)  # next action is determined
    # based on the observation of the state (and the model)
    observation, reward, terminated, truncated, info = env.step(action)
    if terminated:  # this resets the game if we fail (this shouldn't happen
        # after the map is fully known)
        observation, info = env.reset()
env.close()

In [77]:
# Step 1: Define the function `recommend_products`
def recommend_products(user_data, products, n):
    """
    Recommend top n products based on user preferences and past interactions.
    
    Args:
        user_data (dict): User's past ratings and preferences.
        products (list): List of product dictionaries.
        n (int): Number of recommendations to return.
    
    Returns:
        list: Top n recommended products with utility scores.
    """
    # Step 2: Extract user preferences and past ratings
    user_ratings = user_data.get("ratings", {})
    user_preferences = user_data.get("preferences", {})
    
    # Step 3: Initialize an empty list to store utility scores
    utility_scores = []

    # Step 4: Iterate through each product in the products list
    for product in products:
        # Step 5: Calculate the utility score for the current product
        utility_score = calculate_utility(user_ratings, user_preferences, product)
        
        # Step 6: Append the product and its utility score to the utility_scores list
        utility_scores.append((product, utility_score))

    # Step 7: Sort the utility_scores list by score in descending order
    utility_scores.sort(key=lambda x: x[1], reverse=True)

    # Step 8: Return the top n products
    return utility_scores[:n]

# Step 9: Define a helper function `calculate_utility`
def calculate_utility(user_ratings, user_preferences, product):
    """
    Calculate utility score for a product.
    
    Args:
        user_ratings (dict): User's past ratings.
        user_preferences (dict): User's preferences.
        product (dict): Product details.
    
    Returns:
        float: Utility score for the product.
    """
    # Step 10: Implement the scoring logic
    category_score = 1.0 if product["category"] in user_preferences.get("category", []) else 0.5
    
    # If the user has rated this product before, use it; otherwise, take product rating
    rating_score = user_ratings.get(product["id"], product["rating"]) / 5.0  # Normalize rating (0-1)
    
    # Apply a price penalty: Cheaper products are slightly preferred
    price_penalty = 1.0 / (1.0 + (product["price"] / 100.0))  # Normalized price factor
    
    # Combine the scores into a final utility score
    utility = (category_score * 0.4) + (rating_score * 0.5) + (price_penalty * 0.1)
    return utility

# Step 11: Test the function
user_data = {
    "ratings": {"product1": 5, "product2": 3},
    "preferences": {"category": ["electronics", "books"]}
}

products = [
    {"id": "product1", "category": "electronics", "price": 499, "rating": 4.5},
    {"id": "product2", "category": "books", "price": 15, "rating": 4.0},
    {"id": "product3", "category": "clothing", "price": 50, "rating": 3.8},
    {"id": "product4", "category": "electronics", "price": 1000, "rating": 4.7},
    {"id": "product5", "category": "books", "price": 25, "rating": 4.2}
]

# Get top 3 recommendations
recommendations = recommend_products(user_data, products, 3)
for product, score in recommendations:
    print(f"Recommended: {product['id']} | Utility Score: {score:.2f}")


Recommended: product1 | Utility Score: 0.92
Recommended: product5 | Utility Score: 0.90
Recommended: product4 | Utility Score: 0.88
